In [176]:
import re
import os
import chardet
import string
import json
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [177]:
docTextMap = {}
stemmer = PorterStemmer()
applyStem = False

## Task 1

In [178]:
#Used to remove numbers during pre-processing

def is_number(word):
    return word[0].isdigit() or word.isdigit() or is_float(word)

def is_float(word):
    try:
        float_value = float(word)
        return True
    except ValueError:
        return False
def checkForStopWords(word):
    return word.lower() in stopwords
def requireStemming(token, applyStem):
    if applyStem:
        return stemmer.stem(token)
    else:
        return token


In [179]:
stopWordPath = "../Resources/IR_data/AP_DATA/stoplist.txt"

with open(stopWordPath) as file:
    stopwords = file.readlines()
    
    for index, stopword in enumerate(stopwords):
        stopwords[index] = stopword.split("\n")[0]
        
        
punctuations = list(string.punctuation)

extraPunc = ["``", "'s'", "'", "''"]
[punctuations.append(el) for el in extraPunc]

for p in punctuations:
    stopwords.append(p)
        
print(f'Total number of stopwords: {len(stopwords)}')

Total number of stopwords: 531


In [180]:
# Helper method for processing text
def tokenize(text):
    pattern = r"\w+(?:\.?\w)*"
 
    # Tokenize the text based on the pattern
    words = re.findall(pattern, text.lower())

    return words

In [181]:
#Tokenizing, removing stop words and stemming the text
def preprocess(text, applyStem):
    tokens = tokenize(text)
    words = []
    for index, word in enumerate(tokens):
        if not checkForStopWords(word) and not is_number(word):
  #      if word.lower() not in checkForStopWords(word) and not is_number(word):
            words.append(requireStemming(word, applyStem))
    return words


In [182]:
#Map to store docNo and its corresponding text
docTextMap = {}
# Map for storing docId and length of its text
docTextLen = {}

def extractData(path, applyStem):
    doc = re.compile(r'<DOC>(.*?)</DOC>', re.DOTALL)

    with open(path, 'rb') as rawfile:
        result = chardet.detect(rawfile.read())
        encoding = result['encoding']

    with open(path, 'r', encoding=encoding) as file:
        file_content = file.read()
    
    # Read the content of the file
    with open(path, 'r') as file:
        file_content = file.read()

    docs = re.findall(doc, file_content)
    
    for match in docs:
        docNo = re.findall(r'<DOCNO>(.*?)</DOCNO>', match, re.DOTALL)[0]
        text = re.findall(r'<TEXT>(.*?)</TEXT>', match, re.DOTALL)
               
        text = "".join(text).lower()
        
        docTextMap[docNo] = preprocess(text, applyStem)

        docTextLen[docNo] = len(text)

In [183]:
#Retrieving the files from ap98_collection
folder = "../Resources/IR_data/AP_DATA/ap89_collection"

for file in sorted(os.listdir(folder)):
    print(file)
        
    if file != "readme":
        file_path = os.path.join(folder, file)
        extractData(file_path, applyStem)

ap890101
ap890102
ap890103
ap890104
ap890105
ap890106
ap890107
ap890108
ap890109
ap890110
ap890111
ap890112
ap890113
ap890114
ap890115
ap890116
ap890117
ap890118
ap890119
ap890120
ap890121
ap890122
ap890123
ap890124
ap890125
ap890126
ap890127
ap890128
ap890129
ap890130
ap890131
ap890201
ap890202
ap890203
ap890204
ap890205
ap890206
ap890207
ap890208
ap890209
ap890210
ap890211
ap890212
ap890213
ap890214
ap890215
ap890216
ap890217
ap890218
ap890219
ap890220
ap890221
ap890222
ap890223
ap890224
ap890225
ap890226
ap890227
ap890228
ap890301
ap890302
ap890303
ap890304
ap890305
ap890306
ap890307
ap890308
ap890309
ap890310
ap890311
ap890312
ap890313
ap890314
ap890315
ap890316
ap890317
ap890318
ap890319
ap890320
ap890321
ap890322
ap890323
ap890324
ap890325
ap890326
ap890327
ap890328
ap890329
ap890330
ap890401
ap890402
ap890403
ap890404
ap890405
ap890406
ap890407
ap890408
ap890409
ap890410
ap890411
ap890412
ap890413
ap890414
ap890415
ap890416
ap890417
ap890418
ap890419
ap890420
ap890421
ap890422
a

In [184]:
#Get token and term position map
token_index_map = {}
c = 0
token_position_dict = {}

for docNo in docTextMap.keys():
    currentDict = {}
    for index, token in enumerate(docTextMap[docNo]):      
        if token not in currentDict:
            currentDict[token] = []
            
        currentDict[token].append(index + 1)

        if token not in token_index_map:
            c = c + 1
            token_index_map[token] = c
            

    token_position_dict[docNo] = currentDict

In [185]:
def writeFile(data, path):
    if applyStem:
        folderName = "stemmed/"
    else:
        folderName = "unstemmed/"
        
    with open(os.path.join(f'../Resources/{folderName}', path), "w") as file:
        file.write(data)


## Task 2

In [202]:

# Initialize maps for token-index and docID-index
docID_index_map = {}
c = 0

# Read docIDs from file and sort them
with open('..\\Resources\\IR_data\\AP_DATA\doclist.txt', 'r') as file:
    for line in file:
        index, filename = line.split(maxsplit=1)
        index = index.strip()  # Remove leading/trailing whitespace and newline character
        filename = filename.strip()  # Remove leading/trailing whitespace and newline character
        docID_index_map[' '+filename+' '] = index

writeFile(json.dumps(docTextLen), "docTextLen.txt");
writeFile(json.dumps(token_index_map), "tokenIndexMap.txt")

<>:6: SyntaxWarning: invalid escape sequence '\R'
<>:6: SyntaxWarning: invalid escape sequence '\R'
C:\Users\sanje\AppData\Local\Temp\ipykernel_2696\4216683923.py:6: SyntaxWarning: invalid escape sequence '\R'
  with open('..\Resources\IR_data\AP_DATA\doclist.txt', 'r') as file:


In [203]:
# Inverted Index to string, inorder to store it in a text file.
def toString(invertedIndex):
    indexValue = ""

    for key1 in invertedIndex.keys():
        content = f"{key1}:{{"
        inner_content = [
            f"{key2}:[{','.join(map(str, invertedIndex[key1][key2]))}]"
            for key2 in invertedIndex[key1].keys()
        ]
        content += ','.join(inner_content)
        content += "}"
        indexValue += content

    return indexValue



In [204]:
def getCatalogString(str):
    catalog = ""    
    startIndex = 0
    i = 0
    while i < len(str):
        token = ""
        length = 0
        while i < len(str) and str[i] != ':':
            length += 1
            token += str[i]
            i += 1
        while i < len(str) and str[i] != '}':
            length += 1
            i += 1
        i += 1
        catalog += f'{token} {startIndex} {length}\n'
        startIndex += length + 1        
    return catalog

In [205]:
'''def getCatalog(inverted_index):
    # Convert the inverted index map into String
    convertToString = json.dumps(inverted_index)
 
    #Initialise tokens
    tokens = inverted_index.keys()
    catalog = ""
 
    # Looping through the tokens
    for token in tokens:
        #Find the required token
        start = convertToString.find(token) - 1
        #Get the ending of the token
        #If it is the end of the String i.e. -1, then return (length - 1)
        # Else return till "}" is found
        end = len(convertToString) - 1 if convertToString.find("},", start) == -1 else convertToString.find("},", start) + 1
        catalog += f'{token} {start} {end-start+1}\n'
    return catalog '''

'def getCatalog(inverted_index):\n    # Convert the inverted index map into String\n    convertToString = json.dumps(inverted_index)\n \n    #Initialise tokens\n    tokens = inverted_index.keys()\n    catalog = ""\n \n    # Looping through the tokens\n    for token in tokens:\n        #Find the required token\n        start = convertToString.find(token) - 1\n        #Get the ending of the token\n        #If it is the end of the String i.e. -1, then return (length - 1)\n        # Else return till "}" is found\n        end = len(convertToString) - 1 if convertToString.find("},", start) == -1 else convertToString.find("},", start) + 1\n        catalog += f\'{token} {start} {end-start+1}\n\'\n    return catalog '

In [206]:
#Get inverted index
inverted_index = {}

# Get inverted index
docNos = sorted(list(docID_index_map.keys()))

for startIndex in range(0, len(docNos), 1000):
    convert = {}

    endIndex = min(len(docNos) - 1, startIndex + 999)

    for index in range(startIndex, endIndex + 1):
        docNo = docNos[index]

        for token in token_position_dict[docNo]:
            positionList = token_position_dict[docNo][token]

            c = token_index_map[token]
            
            if c not in convert:
                convert[c] = {}
                
            if c not in inverted_index:
                inverted_index[c] = {}
                
            convert[c][docID_index_map[docNo]] = positionList
            inverted_index[c][docID_index_map[docNo]] = positionList

    fileNo = int(startIndex / 1000)
    
    if int(startIndex / 1000) % 20 == 0:
        print(f'Writing into {fileNo}.txt')
        
    # Write inverted index
    s = toString(convert)
    writeFile(s, f'invertedIndex/{fileNo}.txt')
    
    catalogString = getCatalogString(s)
    
    writeFile(catalogString, f'catalogs/{fileNo}.txt')
    


Writing into 0.txt
Writing into 20.txt
Writing into 40.txt
Writing into 60.txt
Writing into 80.txt


In [207]:
# Merging

'''def getCData(path):
    data = {}
    
    with open(path) as file:
        for line in file:
            parts = line.strip().split()
            key = parts[0]
            values = [int(parts[1]), int(parts[2])]
            data[key] = values
        
    return data
'''

'def getCData(path):\n    data = {}\n    \n    with open(path) as file:\n        for line in file:\n            parts = line.strip().split()\n            key = parts[0]\n            values = [int(parts[1]), int(parts[2])]\n            data[key] = values\n        \n    return data\n'

In [208]:
# Converts catalog string to position list
def catalogToPList(catalog):
    positions = {}
    for line in catalog:
        parts = line.split()
        if len(parts) > 1:
            key = parts[0]
            values = [int(parts[1]), int(parts[2])]
            positions[key] = values

    return positions


In [209]:
folder = "stemmed/" if applyStem else "unstemmed/"
# Read catalog file
def openCatalog(path):
    content = ""
    
    with open(f'../Resources/{folder}catalogs/{path}') as file:
        content = file.readlines()
        
    return content


In [210]:
'''def stringToDictionary(inp):
    # Regular expression pattern to match docId and integers
    pattern = r'(\d+):(\[\d+(?:,\d+)*\])'

    # Find all matches in the input string
    matches = re.findall(pattern, inp)

    # Extract docId and integers from matches
    dd = {}
    
    for match in matches:
        doc_id = match[0]
        integers = list(map(int, match[1][1:-1].split(',')))  # Extract integers within []
        dd[doc_id] = integers
        
        
    return dd '''

'''def stringToDictionary(str):
    # Regular expression pattern to match docId and integers
    p = r'(\d+):(\[\d+(?:,\d+)*\])'

    # Find all matches in the input string
    match = re.findall(p, str)

    # Extract docId and integers from matches
    extractedData = {doc_id: [int(n) for n in integers.split(',')] for doc_id, integers in match}
    
    return extractedData '''

def stringToDictionary(str):
    # Regular expression pattern to match docId and integers
    p = r'(\d+):(\[\d+(?:,\d+)*\])'

    # Find all matches in the input string
    match = re.finditer(p, str)

    # Extract docId and integers from matches
    extractedData = {}
    for match in match:
        doc_id = match.group(1)
        nums = list(map(int, match.group(2)[1:-1].split(',')))  # Extract integers within []
        extractedData[doc_id] = nums
        
    return extractedData



<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sanje\AppData\Local\Temp\ipykernel_2696\1786342602.py:1: SyntaxWarning: invalid escape sequence '\d'
  '''def stringToDictionary(inp):
C:\Users\sanje\AppData\Local\Temp\ipykernel_2696\1786342602.py:19: SyntaxWarning: invalid escape sequence '\d'
  '''def stringToDictionary(str):


In [211]:
# Reads string from inverted Index
'''def extractDict(key, position, path):
    start, length = position
    
    # read content from file on specific start, end index
    with open(f'../Resources/{folder}/invertedIndex/{path}') as file:
        file.seek(start)
        line = file.read(length + 1)
        
    token = ""
    i = 0

    while i < len(line) and line[i] != ':':
        token += line[i]
        i += 1
        
    dd = stringToDictionary(line[len(token)+2:-1])
    updatedDD = {}
    
    updatedDD[token] = dd
    
    return updatedDD '''

def extractDict(key, position, path):
    start, length = position
    
    # Read content from file with specific start and length
    with open(f'../Resources/{folder}/invertedIndex/{path}') as file:
        file.seek(start)
        line = file.read(length + 1)
        
    token = line.split(":")[0]

    # Extract the catalog string
    catalog_string = line[len(token)+2:-1]

    # Convert catalog string to dictionary
    dict = stringToDictionary(catalog_string)
    
    # Create a dictionary with the token as key and the converted catalog as value
    finalDict = {token: dd}
    
    return finalDict


In [212]:
# Returns merged dictionaries based on catalogs, and index files.
def mergeDict(c1, c2, i1, i2):
    # Gets catalog data in string
    catalog1 = openCatalog(c1)
    catalog2 = openCatalog(c2)
    
    # Converts catalog string to keys and list of [start, length]
    catalog1 = catalogToPList(catalog1)
    catalog2 = catalogToPList(catalog2)
    
    finalDict = {}

    # Loop over catalog1 keys, include which are part of both catalog1
    # and catalog2
    for k1 in catalog1.keys():
        dict1 = extractDict(k1, catalog1[k1], i1)

        if k1 in catalog2.keys():
            dict2 = extractDict(k1, catalog2[k1], i2)
            dict1[k1].update(dict2[k1])
        
        mergedDict[k1] = dict1[k1]

    # Loop over catalog2 keys, include which are not part of catalog1
    for k2 in catalog2.keys():
        dict2 = extractDict(k2, catalog2[k2], i2)

        if k2 not in catalog1.keys():
            finalDict[k2] = dict2[k2]
            
    mergedDictStr = toString(mergedDict)
    return mergedDict, getCatalogString(mergedDictStr)

In [213]:
def getNumber(file):
    # Extract the numeric part of the file name using regular expression
    match = re.match(r'(\d+)\.txt', file)
    return int(match.group(1)) if match else float('inf')

def performMerge(indexFile, catalogFile, mergeFile, file1, file2):
    merged_dict, catalog = mergeDict(indexFile[file1], indexFile[file2], catalogFile[file1], catalogFile[file2])
    merged_string = toString(merged_dict)
    writeFile(merged_string, f'InvertedIndex/{mergeFile}.txt')
    writeFile(catalog, f'catalogs/{mergeFile}.txt')

    

In [214]:
'''def mergeFiles(indexPath, catalogPath):
    indexFileList = sorted(os.listdir(indexPath), key=getNumber)
    catalogFileList = sorted(os.listdir(catalogPath), key=getNumber)
    
    indexFileList = [name for name in indexFileList if name.endswith('.txt')]
    catalogFileList = [name for name in catalogFileList if name.endswith('.txt')]
    
    n = len(indexFileList)
    
    while n > 1:
        now = time.time()
        print("\n************************************\n")
        print(f'n={n}')
        
        for i in range(0, n - 1, 2):
            performMerge(indexFileList, catalogFileList, i//2, i, i + 1)

        if n % 2:
            performMerge(indexFileList, catalogFileList, (n//2)-1, (n//2)-1, n-1)

        n //= 2 '''

def merge(indexPath, catalogPath):
    indexFileList = sorted(os.listdir(indexPath), key=getNumber)
    catalogFileList = sorted(os.listdir(catalogPath), key=getNumber)
    
    indexFileList = [name for name in indexFileList if name.endswith('.txt')]
    catalogFileList = [name for name in catalogFileList if name.endswith('.txt')]
    
    n = len(indexFileList)
    
    while n > 1:
        print("\n************************************\n")
        print(f'n={n}')
        
        for i in range(0, n - 1, 2):
            performMerge(indexFileList, catalogFileList, i//2, i, i + 1)
    
        if n % 2:
            performMerge(indexFileList, catalogFileList, (n//2)-1, (n//2)-1, n-1)
    
        n //= 2


    




        
merge(f'../Resources/{folder}InvertedIndex', f'../Resources/{folder}catalogs')



************************************

n=85

************************************

n=42

************************************

n=21

************************************

n=10

************************************

n=5

************************************

n=2


## Compression

In [217]:
#Stemmed Compression

import gzip
import shutil

inPath = f'../Resources/stemmed/invertedIndex/0.txt'
outPath = f'../Resources/stemmed/invertedIndex/0-compressed.txt'

def compress_file(inPath, outPath):
    with open(inPath, 'rb') as f_in:
        with gzip.open(outPath, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
compress_file(inPath, outPath)

def decompress_file(inPath, outPath):
    with gzip.open(inPath, 'rb') as f_in:
        with open(outPath, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

decompress_file(outPath, f'../Resources/stemmed/invertedIndex/0-decompressed.txt')


In [218]:
#UnStemmed Compression

import gzip
import shutil

inPath = f'../Resources/unstemmed/invertedIndex/0.txt'
outPath = f'../Resources/unstemmed/invertedIndex/0-compressed.txt'

def compress_file(inPath, outPath):
    with open(inPath, 'rb') as f_in:
        with gzip.open(outPath, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
compress_file(inPath, outPath)

def decompress_file(inPath, outPath):
    with gzip.open(inPath, 'rb') as f_in:
        with open(outPath, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

decompress_file(outPath, f'../Resources/unstemmed/invertedIndex/0-decompressed.txt')